In [ ]:
import time
import requests
import json

def fetch_schedule():
    schedule_url = "https://statsapi.mlb.com/api/v1/schedule?sportId=1&season=2024"
    response = requests.get(schedule_url)
    return response.json()

def detect_updates(current_data, new_data):
    if current_data != new_data:
        print("New data detected!")
        # Process the updated data here
        return new_data
    return current_data

# Polling loop
def start_polling(interval=60):  # Check every 60 seconds
    print("Starting polling...")
    current_data = fetch_schedule()

    while True:
        time.sleep(interval)
        try:
            new_data = fetch_schedule()
            new_data['new_data'] = "New Data"
            current_data = detect_updates(current_data, new_data)
        except Exception as e:
            print(f"Error fetching data: {e}")

start_polling()


Starting polling...
New data detected!


In [1]:
from utils import process_endpoint_url
import pandas as pd
import json
import requests

class DataIngestion:
    def __init__(self):
        pass

    def initiate_data_ingestion(self):
        game_pk = self.latest_completed_game()
        game_data = self.get_single_game_data(game_pk)
        return game_data

    def latest_completed_game(self):
        schedule_endpoint_url = 'https://statsapi.mlb.com/api/v1/schedule?sportId=1&season=2024'
        
        # Fetch schedule data
        schedule_dates = process_endpoint_url(schedule_endpoint_url, "dates")

        # Normalize games data into a DataFrame
        games = pd.json_normalize(
            schedule_dates.explode('games').reset_index(drop=True)['games']
        )

        date_columns = [
            "gameDate",
            "officialDate",
            "rescheduleDate",
            "rescheduleGameDate",
            "rescheduledFromDate",
            "resumeDate",
            "resumeGameDate",
            "resumedFromDate"
        ]

        # Convert the specified columns to datetime
        for col in date_columns:
            games[col] = pd.to_datetime(games[col], errors='coerce')

        # Filter for completed games
        completed_games = games[
            games['status.detailedState'].isin(['Final', 'Completed Early'])
        ]

        # Get the most recent completed game
        completed_games = completed_games.sort_values(by='gameDate', ascending=False)
        latest_game = completed_games.iloc[0]

        return latest_game['gamePk']
    
    
    def get_single_game_data(self, game_pk):
        single_game_feed_url = f'https://statsapi.mlb.com/api/v1.1/game/{game_pk}/feed/live'

        single_game_info_json = json.loads(requests.get(single_game_feed_url).content)

        return single_game_info_json
    
    def get_games_between_dates(self, start_date, end_date):
        """
        Fetches all games and filters them between the specified start and end dates.

        Args:
            start_date (str): The start date in the format 'YYYY-MM-DD'.
            end_date (str): The end date in the format 'YYYY-MM-DD'.

        Returns:
            DataFrame: A DataFrame containing games between the specified dates.
        """
        schedule_endpoint_url = 'https://statsapi.mlb.com/api/v1/schedule?sportId=1&season=2024'

        # Fetch all schedule data
        schedule_dates = process_endpoint_url(schedule_endpoint_url, "dates")

        # Normalize games data into a DataFrame
        games = pd.json_normalize(
            schedule_dates.explode('games').reset_index(drop=True)['games']
        )

        date_columns = [
            "gameDate",
            "officialDate",
            "rescheduleDate",
            "rescheduleGameDate",
            "rescheduledFromDate",
            "resumeDate",
            "resumeGameDate",
            "resumedFromDate"
        ]

        # Convert relevant columns to datetime
        for col in date_columns:
            games[col] = pd.to_datetime(games[col], errors='coerce')

        # Filter games by 'gameDate' within the specified range
        start_date = pd.to_datetime(start_date).tz_localize('UTC')
        end_date = pd.to_datetime(end_date).tz_localize('UTC')

        filtered_games = games[
            (games['gameDate'] >= start_date) & (games['gameDate'] <= end_date)
        ]

        return filtered_games

    

data_ingestion = DataIngestion()
game_data = data_ingestion.initiate_data_ingestion()



In [8]:
game_data['gameData']['teams']['home']['name']

'New York Yankees'

In [35]:
start_date = pd.to_datetime(start_date).tz_localize('UTC')
end_date = pd.to_datetime(end_date).tz_localize('UTC')
print(start_date)


2024-04-01 00:00:00+00:00


In [39]:
data_ingestion = DataIngestion()
game_data = data_ingestion.initiate_data_ingestion()


In [44]:
print((game_data["gameData"]["datetime"]["dateTime"]))

2024-10-31T00:08:00Z


In [43]:
print(pd.to_datetime(game_data["gameData"]["datetime"]["dateTime"]).tz_convert('UTC'))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [7]:
from google import genai
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("API_KEY")

def generate_text(prompt):
    client = genai.Client(api_key=API_KEY, http_options={'api_version': 'v1alpha'})
    text = ""
    for chunk in client.models.generate_content_stream(
        model='gemini-2.0-flash-thinking-exp', contents=prompt
    ):
        for part in chunk.candidates[0].content.parts:
            if not part.thought:
                text += part.text
    return text

generate_text("What is genAI")

'**GenAI, short for Generative AI, refers to a category of artificial intelligence algorithms and models that are designed to *generate* new content, rather than simply analyzing or acting upon existing data.**\n\nThink of traditional AI as being good at tasks like:\n\n* **Classification:** Identifying if an image is a cat or a dog.\n* **Prediction:**  Predicting stock prices based on historical data.\n* **Automation:**  Automating repetitive tasks based on predefined rules.\n\n**GenAI, on the other hand, is about creation.** It\'s about building AI that can produce things that resemble human creativity and output, such as:\n\n* **Text:** Writing articles, poems, code, scripts, emails, chat responses, and more.\n* **Images:** Creating realistic photos, artwork, illustrations, logos, and even manipulating existing images.\n* **Audio:** Generating music, speech, sound effects, and voiceovers.\n* **Video:** Producing animations, short clips, and even attempting to create longer video cont

In [39]:
from datetime import datetime

date_str = "2024-10-31T00:08:00Z"
date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")  # Parse full datetime
date_only = date_obj.date()  # Extract only the date part

date_only

datetime.date(2024, 10, 31)

In [43]:
from datetime import datetime, timezone

date_str = "2024-10-31T00:08:00Z"
date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")  # Parse full datetime
date_only = date_obj.date()  # Extract only the date part

start = "2024-10-31"
start_date = datetime.strptime(start, "%Y-%m-%d").date()  # Convert string to date

print(date_only, start_date)
print(date_only == start_date)  # Output: True


2024-10-31 2024-10-31
True


In [40]:
start = "2024-10-31"
datetime.strptime(start, "%Y-%m-%d").date()


datetime.date(2024, 10, 31)

In [41]:
date_only == start

False

In [23]:
start = "2024-10-31"
datetime.strptime(start, "%Y-%m-%d").replace(tzinfo=timezone.utc).date()

datetime.date(2024, 10, 31)

In [10]:
import datetime

In [11]:
json_data = {'title': 'Dodgers Edge Out Yankees in Thrilling Subway Series Showdown!', 'sections': [{'heading': 'Introduction', 'content': 'Yankee Stadium was electric as the Los Angeles Dodgers and New York Yankees clashed in a highly anticipated matchup. With partly cloudy skies, a comfortable 67 degrees, and a gentle breeze blowing out to left field, the stage was perfectly set for a classic baseball battle. Fans were on the edge of their seats, and neither team disappointed, delivering a back-and-forth thriller that kept everyone guessing until the final out.'}, {'heading': 'Game Highlights', 'content': "The game started with a bang in the bottom of the first. Yankees' slugger Aaron Judge launched a two-run homer to right-center, bringing Juan Soto home and sending the Yankee faithful into a frenzy.  Moments later, Jazz Chisholm Jr. followed suit with a solo shot to right-center, extending the Yankees' early lead to 3-0.  In the second inning, Anthony Volpe doubled, advanced to third, and then Alex Verdugo singled sharply to right, scoring Volpe and making it 4-0 Yankees.  Giancarlo Stanton joined the home run derby in the third, belting a solo homer to right field, pushing the lead to a seemingly comfortable 5-0. However, the Dodgers were far from done.  The top of the fifth inning saw a dramatic shift. Enrique Hernández singled, and a fielding error put runners on base.  Mookie Betts then grounded a single to first, scoring Hernández and initiating the rally. Freddie Freeman followed with a clutch line drive single to center, driving in two more runs.  The momentum fully swung when Teoscar Hernández ripped a double to center, plating Betts and Freeman and suddenly giving the Dodgers a 7-5 lead!  This explosive inning completely changed the complexion of the game.  The Yankees responded in the bottom of the sixth when Giancarlo Stanton hit a sacrifice fly to center, scoring Juan Soto and narrowing the gap to 7-6.  In the eighth, the Dodgers added insurance runs via sacrifice flies from Gavin Lux and Mookie Betts, each driving in a run. These runs proved vital as the Yankees couldn't muster another comeback, sealing a hard-fought 7-6 victory for Los Angeles."}, {'heading': 'Key Players', 'content': "For the Dodgers, Mookie Betts was instrumental, driving in crucial runs and consistently finding ways to get on base, showcasing his leadoff hitter prowess. Freddie Freeman delivered a timely two-RBI single during the pivotal fifth-inning rally, demonstrating his clutch hitting ability. Teoscar Hernández's double was arguably the biggest hit of the game, driving in two runs and completing the Dodgers' comeback.  On the Yankees side, Aaron Judge provided the initial spark with his two-run home run and later added a double, proving his status as a premier power hitter. Giancarlo Stanton contributed significantly with a home run and a sacrifice fly, accounting for two of the Yankees' six runs and displaying his consistent offensive threat. Jazz Chisholm Jr.'s early solo home run also added to the Yankees' initial surge, highlighting his ability to change the game with one swing."}], 'links': [{'Aaron Judge homers': 'https://www.mlb.com/video/search?q=playid="08286940-26d1-411d-b53c-1897085657ac"'}, {'Jazz Chisholm Jr. homers': 'https://www.mlb.com/video/search?q=playid="b21f2da5-a381-40c4-8141-f7f451ae01da"'}, {'Alex Verdugo RBI Single': 'https://www.mlb.com/video/search?q=playid="b1d9e59d-24ae-4ca4-bf1f-a8c590d0cf05"'}, {'Giancarlo Stanton solo homer': 'https://www.mlb.com/video/search?q=playid="01d4ea9b-8945-425c-94a1-374cb051b7fb"'}, {'Mookie Betts RBI Single': 'https://www.mlb.com/video/search?q=playid="09d1035c-1e51-4905-b163-1dfe21f16fec"'}, {'Freddie Freeman 2-RBI Single': 'https://www.mlb.com/video/search?q=playid="fba35e51-ca2c-4111-a61c-11300222f434"'}, {'Teoscar Hernández 2-RBI Double': 'https://www.mlb.com/video/search?q=playid="f51badfa-6a51-4045-8a93-a2b2d49e0ad6"'}, {'Giancarlo Stanton Sac Fly RBI': 'https://www.mlb.com/video/search?q=playid="f40d8e7a-14fe-415d-8dd9-63af4ed9d519"'}, {'Gavin Lux Sac Fly RBI': 'https://www.mlb.com/video/search?q=playid="92781ed2-1ad7-41cd-844a-d8c1a1451b13"'}, {'Mookie Betts Sac Fly RBI': 'https://www.mlb.com/video/search?q=playid="2137382a-1047-4bee-b27a-da06e0909bda"'}], 'conclusion': "This Dodgers-Yankees clash was a true testament to baseball's excitement, showcasing power hitting and dramatic comebacks. The Dodgers demonstrated resilience to overcome an early deficit and secure a thrilling 7-6 victory on the road.  Relive all the key moments by clicking the video highlights above and share your thoughts on this instant classic in the comments below!  Stay locked in for more MLB action!", 'created_at': datetime.datetime(2025, 2, 1, 5, 29, 46, 852597), 'game_date': '2024-10-31', 'team_home': 'New York Yankees', 'team_away': 'Los Angeles Dodgers'}

In [12]:
print(json_data)

{'title': 'Dodgers Edge Out Yankees in Thrilling Subway Series Showdown!', 'sections': [{'heading': 'Introduction', 'content': 'Yankee Stadium was electric as the Los Angeles Dodgers and New York Yankees clashed in a highly anticipated matchup. With partly cloudy skies, a comfortable 67 degrees, and a gentle breeze blowing out to left field, the stage was perfectly set for a classic baseball battle. Fans were on the edge of their seats, and neither team disappointed, delivering a back-and-forth thriller that kept everyone guessing until the final out.'}, {'heading': 'Game Highlights', 'content': "The game started with a bang in the bottom of the first. Yankees' slugger Aaron Judge launched a two-run homer to right-center, bringing Juan Soto home and sending the Yankee faithful into a frenzy.  Moments later, Jazz Chisholm Jr. followed suit with a solo shot to right-center, extending the Yankees' early lead to 3-0.  In the second inning, Anthony Volpe doubled, advanced to third, and then

In [ ]:
{
    "title": "arsga",
    "team_away": "argaer",
    "team_home": "aerg",
    "game_date": "2025-02-01",
    "sections": [
        {
            "heading": "arg",
            "content": "aegr"
        },
        {
            "heading": "aerg",
            "content": "aeerg"
        }
    ],
    "conclusion": "aergaerg"
}

In [ ]:
{
    "title": "Dodgers Edge Out Yankees in Thrilling 7-6 Slugfest at Yankee Stadium",
    "sections": [
        {
            "heading": "Introduction",
            "content": "Under partly cloudy skies and a comfortable 67 degrees, with a gentle 5 mph breeze blowing out to left field, Yankee Stadium was the stage for a classic Los Angeles Dodgers versus New York Yankees showdown. This wasn't just any game; it was a back-and-forth battle that kept fans on the edge of their seats until the very last out.  Both teams brought their offensive firepower, resulting in a high-scoring affair that ultimately saw the Dodgers narrowly escape with a 7-6 victory over the Bronx Bombers. From explosive home runs to clutch hits, this game had it all, showcasing the best of baseball's drama and excitement."
        },
        {
            "heading": "Game Highlights",
            "content": "The game started with a bang in the bottom of the first inning as Aaron Judge launched a two-run homer to right-center, immediately putting the Yankees ahead. Not to be outdone, Jazz Chisholm Jr. followed up with a solo shot to the same part of the park, extending the Yankees' lead to 3-0 before many fans had even settled into their seats. The Yankees continued to apply pressure in the second when Alex Verdugo lined a single to right, scoring Anthony Volpe and making it 4-0. Giancarlo Stanton joined the home run party in the third, belting a solo homer to right field, seemingly giving the Yankees a commanding 5-0 advantage. \n\nHowever, the Dodgers, known for their resilience, began their comeback in the fifth inning. Mookie Betts ignited the rally with a ground ball single that scored Enrique Hernández, cutting into the lead. Freddie Freeman followed with a line drive single to center, bringing home Tommy Edman and Will Smith, further narrowing the gap. The inning culminated with Teoscar Hernández ripping a sharp double to center, scoring both Betts and Freeman, and incredibly, the Dodgers had surged ahead 5-4.  \n\nThe Yankees responded swiftly in the sixth when Giancarlo Stanton drove in Juan Soto with a sacrifice fly to center, tying the game once again at 5-5.  The tension mounted as the game progressed into the later innings. In the eighth, the Dodgers manufactured runs with sacrifice flies from Gavin Lux and Mookie Betts, each scoring Enrique Hernández and Tommy Edman respectively, to regain a 7-5 lead.  The Yankees weren't done yet. In the bottom of the eighth, Aaron Judge doubled sharply to left, driving in a run and pulling the Yankees within one, 7-6. Despite threatening, the Yankees couldn't complete the comeback, as the Dodgers' pitching held firm in the ninth, securing a hard-fought 7-6 victory in a game filled with twists and turns."
        },
        {
            "heading": "Key Players",
            "content": "For the Dodgers, Mookie Betts was a standout performer, driving in crucial runs with a single and a sacrifice fly, consistently putting himself in scoring positions and making plays happen. Freddie Freeman also had a significant impact with his key RBI single during the Dodgers' fifth-inning rally. Teoscar Hernández's timely double drove in two crucial runs, proving to be a game-changer. On the Yankees side, Aaron Judge was undeniably the offensive powerhouse, launching a two-run homer and a run-scoring double, along with drawing walks and consistently being a threat at the plate. Giancarlo Stanton contributed significantly with two home runs and a sacrifice fly, driving in a total of three runs and showcasing his power. Jazz Chisholm Jr.'s leadoff home run set an early energetic tone for the Yankees, demonstrating his ability to spark the offense. Alex Verdugo's RBI single also played a part in the Yankees' early lead, highlighting his contribution to the team's offensive efforts."
        }
    ],
    "links": [
        {
            "Aaron Judge's Opening Homer": "https://www.mlb.com/video/search?q=playid=\"08286940-26d1-411d-b53c-1897085657ac\"",
            "Chisholm Jr.'s Follow-Up Home Run": "https://www.mlb.com/video/search?q=playid=\"b21f2da5-a381-40c4-8141-f7f451ae01da\"",
            "Verdugo Extends Yankees Lead": "https://www.mlb.com/video/search?q=playid=\"b1d9e59d-24ae-4ca4-bf1f-a8c590d0cf05\"",
            "Stanton's Solo Shot": "https://www.mlb.com/video/search?q=playid=\"01d4ea9b-8945-425c-94a1-374cb051b7fb\"",
            "Betts Starts Dodgers Rally": "https://www.mlb.com/video/search?q=playid=\"09d1035c-1e51-4905-b163-1dfe21f16fec\"",
            "Freeman Ties the Game": "https://www.mlb.com/video/search?q=playid=\"fba35e51-ca2c-4111-a61c-11300222f434\"",
            "Teoscar Hernández Gives Dodgers the Lead": "https://www.mlb.com/video/search?q=playid=\"f51badfa-6a51-4045-8a93-a2b2d49e0ad6\"",
            "Stanton's Sacrifice Fly Ties It Up": "https://www.mlb.com/video/search?q=playid=\"f40d8e7a-14fe-415d-8dd9-63af4ed9d519\"",
            "Lux Sac Fly Nabs Lead": "https://www.mlb.com/video/search?q=playid=\"92781ed2-1ad7-41cd-844a-d8c1a1451b13\"",
            "Betts Extends Lead with Sac Fly": "https://www.mlb.com/video/search?q=playid=\"2137382a-1047-4bee-b27a-da06e0909bda\""
        }
    ],
    "conclusion": "What a game! The Dodgers and Yankees delivered a true baseball spectacle, packed with home runs, clutch hits, and nail-biting tension.  Ultimately, the Dodgers emerged victorious in this high-scoring contest, but both teams showcased their offensive firepower.  If you missed any of the action, be sure to check out the video highlights above and relive the excitement.  Let us know in the comments what your favorite moment of this thrilling game was!",
    "game_date": "2024-10-31",
    "team_home": "New York Yankees",
    "team_away": "Los Angeles Dodgers"
}